### Question to ask:
1. when cleaning email address, I can check if each email address contain virginia to filter out all the emial that are computing id only and entered gmail instead of virginia.edu. However, for all 2 do file, there is one cell has wrong computing id, how to find that cell without hard coding?

2. while delete duplicate, the originial stata do file drop the later one(according to the start date), however, the privious row has more NA than the one we need to delete, is there any rules we need to follow when we delete duplicate?

3. how to deal with label, pandas dataframe do not have such functionality to assign a label to a cell

In [1]:
#this is the python script try to replicate the 2a stata do file
import pandas as pd
import numpy as np
from utils import cleanColumns

In [2]:
#define a function to convert columns name to intutitive column names
def convert_cols(df):
    df=df.rename({
                        'q1':'email',
                        'q4_1':'sim_fbsk',
                        'q5':'sim_txt_fbsk',
                        'q6_1':'sim_cmsk',
                        'q7':'sim_txt_cmsk',
                        'q8':'sim_txt_eth_behavior',
                        'q9':'sim_txt_eth_contribute',
                        'q10_1':'beh_fidgeting',
                        'q10_2':'beh_humming',
                        'q10_3':'beh_excitable',
                        'q10_4':'beh_inattentive',
                        'q10_5':'beh_short_attention',
                        'q10_6':'beh_quarrelsome',
                        'q10_7':'beh_acts_smart',
                        'q10_8':'beh_unpredictable',
                        'q10_9':'beh_defiant',
                        'q10_10':'beh_uncooperative',
                        'q10_11':'beh_easily_frustrated',
                        'q10_12':'beh_disturbs_others',
                        'q10_13':'beh_restless',
                        'q10_14':'beh_mood_changes',
                        'q11_1':'app_coach_stu',
                        'q12_1':'app_adjust_expect',
                        'q13_1':'app_guidance_couns',
                        'q14_1':'app_rec_sped',
                        'q15_1':'app_discp_refer',
                        'q16_1':'app_confer_stu',
                        'q17_1':'app_confer_parent',
                        'q18_1':'app_behavior_plan',
                        'q19_1':'app_challenge_work',
                        'q20_1':'app_spend_time',
                        'q21_1':'app_space_regroup',
                        'q22_1':'app_beh_manage_coach',
                        'q23_1':'app_beh_manage_teach',
                        'q24':'sim_txt_supports',
                        'q25_1':'sim_nervous',
                        'q25_2':'sim_beneficial',
                        'q25_3':'sim_worried_perform',
                        'q25_4':'sim_useful_tool',
                        'q25_5':'sim_relevant_studies',
                        'q25_6':'sim_relevant_prof',
                        'q25_7':'sim_like_use_again',
                        'q25_8':'sim_recommend',
                        'q25_9':'sim_sufficient_prep',
                        'q25_10':'sim_enough_time',
                        'q26':'sim_txt_beneficial',
                        'q27':'sim_txt_improve_exp',
                        'q28':'sim_txt_concerns' 
                       },axis=1)
    return df

In [3]:
#function that fixed the email address to correct format
def fix_email_add(string):
    result=''
    index=string.find("@")
    if index==-1:
        result=string+'@virginia.edu'
    else:
        result=string[:index]+'@virginia.edu'
    return result 

In [4]:
def format_email(data):
    data.email =data.email.str.strip()
    data.email =data.email.str.lower()
    email_fix = data.loc[~data["email"].str.contains("virginia")]
    email_fix['email']=email_fix['email'].apply(lambda x:fix_email_add(x))
    for i in email_fix.index:
        data.at[i,"email"]=email_fix.at[i,'email']
    return data
    

In [5]:
#create map for beh columns and sim columns

def generate_Iowa_Score_Scale(data,beh_map,sim_map,reverse_beh,reverse_sim):
    data.loc[:,'beh_fidgeting':'beh_mood_changes']=data.loc[:,'beh_fidgeting':'beh_mood_changes'].replace(beh_map)
    data.loc[:,'sim_nervous' :'sim_enough_time']=data.loc[:,'sim_nervous' :'sim_enough_time'].replace(sim_map)
    data.loc[:,'app_coach_stu':'app_beh_manage_teach']=data.loc[:,'app_coach_stu':'app_beh_manage_teach'].astype('float')
    data[['sim_fbsk','sim_cmsk']]=data[['sim_fbsk','sim_cmsk']].astype('float')
    data['beh_rating_opdefiant']=data.loc[:,'beh_quarrelsome': 'beh_uncooperative'].mean(axis=1)
    data['beh_rating_impulsive']=data.loc[:,'beh_fidgeting':'beh_short_attention'] .mean(axis=1)
    data['beh_rating'] =data.loc[:,'beh_fidgeting':'beh_mood_changes'].mean(axis=1)
    #label var beh_rating_opdefiant "Iowa Connors Operational Defiant"
    #label var beh_rating_impulsive "Iowa Connors Impulsive"
    #label var beh_rating "Iowa Connors Overall"
    

In [6]:
#Reverse coding management approaches scale 
def Reverse_Approach_Scale(data,app_map):
    app_columns=[x for x in data.columns if x.startswith("app")]
    for x in app_columns:
        name_rc=x+'_rc'
        data[name_rc]=data[x].replace(app_map)

In [7]:
def Generate_App_Scale(data):
    data['manage_app_negative']= data.loc[:,'app_coach_stu':'app_discp_refer'].mean(axis=1)
    data['manage_app_positive']=data.loc[:,'app_confer_stu':'app_beh_manage_teach'].mean(axis=1)

In [8]:
#load the file from data
data = pd.read_csv("../data/Post-Simulator Baseline Student Survey- Summer 2018_January 22, 2020_16.58.csv",skiprows=[1,2])
data.head()

,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,...,Q25_4,Q25_5,Q25_6,Q25_7,Q25_8,Q25_9,Q25_10,Q26,Q27,Q28
0,2018-05-30 12:23:22,2018-05-30 12:24:48,Survey Preview,NaN,100,86,True,2018-05-30 12:24:49,R_1opCvzbgQYDLHKN,NaN,...,Somewhat agree,Strongly agree,Somewhat agree,Undecided,Somewhat disagree,Strongly disagree,Somewhat disagree,NaN,NaN,NaN
1,2018-05-30 13:30:45,2018-05-30 13:32:13,Survey Preview,NaN,100,87,True,2018-05-30 13:32:13,R_1JEm2v4BI4ajTNP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-06-08 07:18:04,2018-06-08 07:36:22,IP Address,199.111.228.147,100,1097,True,2018-06-08 07:36:22,R_ZKuGL3K4Mp1Gked,NaN,...,Strongly agree,Strongly agree,Strongly agree,Strongly agree,Strongly agree,Somewhat agree,Strongly agree,It felt like an actual classroom\n\nThe differ...,It was sort of hard to understand what they we...,none
3,2018-06-08 07:57:16,2018-06-08 08:09:18,IP Address,199.111.228.147,100,722,True,2018-06-08 08:09:18,R_SCWEWpvi6QkN4U9,NaN,...,Strongly agree,Somewhat agree,Somewhat agree,Strongly agree,Strongly agree,Somewhat agree,Somewhat agree,Being able to try out certain ways of speaking...,Increased volume and vocal clarity.,None.
4,2018-06-08 08:19:08,2018-06-08 08:42:41,IP Address,199.111.228.147,100,1412,True,2018-06-08 08:42:41,R_1kMOho2B2VK5Hkr,NaN,...,Strongly agree,Strongly agree,Strongly agree,Strongly agree,Strongly agree,Somewhat agree,Somewhat agree,Thinking about wait time. I talked over the s...,I could use help developing ice breakers. I a...,None at this time


In [9]:
data.columns = cleanColumns(data.columns)
#drop unnecessary observation
data = data[data['q1'].notna()]
reverse_beh = {1:"Not at all", 2: "Just a little", 3: "Pretty much" ,4: "Very much"}
reverse_sim = {5: "Strongly agree", 4:"Somewhat agree", 3: "Undecided", 2: "Somewhat disagree", 1: "Strongly disagree"}
beh_map={"Not at all":1,  "Just a little":2,  "Pretty much":3 , "Very much":4}
sim_map={"Strongly agree":5, "Somewhat agree":4, "Undecided":3, "Somewhat disagree":2, "Strongly disagree":1}
app_map = {1:10, 2:9, 3:8,4:7,5:6,6:5,7:4,8:3,9:2,10:1}

In [10]:
#drop unnecessary variables
unused_columns=['enddate', 'status', 'ipaddress', 'progress',
       'duration (in seconds)', 'finished', 'recordeddate', 'responseid',
       'recipientlastname', 'recipientfirstname', 'recipientemail',
       'externalreference', 'locationlatitude', 'locationlongitude',
       'distributionchannel', 'userlanguage']
data=data.drop(columns=unused_columns)


In [11]:
#rename the columns to intuitive names
data=convert_cols(data)
data.columns

Index(['startdate', 'email', 'sim_fbsk', 'affect', 'sim_cmsk', 'score',
       'rationale', 'sim_txt_eth_contribute', 'beh_fidgeting', 'beh_humming',
       'beh_excitable', 'beh_inattentive', 'beh_short_attention',
       'beh_quarrelsome', 'beh_acts_smart', 'beh_unpredictable', 'beh_defiant',
       'beh_uncooperative', 'beh_easily_frustrated', 'beh_disturbs_others',
       'beh_restless', 'beh_mood_changes', 'app_coach_stu',
       'app_adjust_expect', 'app_guidance_couns', 'app_rec_sped',
       'app_discp_refer', 'app_confer_stu', 'app_confer_parent',
       'app_behavior_plan', 'app_challenge_work', 'app_spend_time',
       'app_space_regroup', 'app_beh_manage_coach', 'app_beh_manage_teach',
       'sim_txt_supports', 'sim_nervous', 'sim_beneficial',
       'sim_worried_perform', 'sim_useful_tool', 'sim_relevant_studies',
       'sim_relevant_prof', 'sim_like_use_again', 'sim_recommend',
       'sim_sufficient_prep', 'sim_enough_time', 'sim_txt_beneficial',
       'sim_txt_improv

In [12]:
data=format_email(data)

<ipython-input-4-c46ddd917ce3>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  email_fix['email']=email_fix['email'].apply(lambda x:fix_email_add(x))


In [13]:
#drop duplicate


In [14]:
#drop startdate
data=data.drop(columns=['startdate'])

In [15]:
generate_Iowa_Score_Scale(data,beh_map,sim_map,reverse_beh,reverse_sim)
data


,email,sim_fbsk,affect,sim_cmsk,score,rationale,sim_txt_eth_contribute,beh_fidgeting,beh_humming,beh_excitable,...,sim_like_use_again,sim_recommend,sim_sufficient_prep,sim_enough_time,sim_txt_beneficial,sim_txt_improve_exp,sim_txt_concerns,beh_rating_opdefiant,beh_rating_impulsive,beh_rating
2,gct2y@virginia.edu,6.0,A few of the students seemed a bit confused ab...,5.0,"The students provided great ideas, but even th...","Ethan seemed a little bit too cool for school,...","I've seen students like Ethan, and the most su...",3.0,2.0,2.0,...,5.0,5.0,4.0,5.0,It felt like an actual classroom\n\nThe differ...,It was sort of hard to understand what they we...,none,1.0,2.6,1.642857
3,wmn6kc@virginia.edu,7.0,I think my feedback allowed everyone to voice ...,10.0,"I asked the students as a whole for ideas, and...",Mostly on task. He did go text his mother befo...,Most likely it would have to do with his famil...,2.0,1.0,1.0,...,5.0,5.0,4.0,4.0,Being able to try out certain ways of speaking...,Increased volume and vocal clarity.,None.,1.0,1.2,1.071429
4,kby6hf@virginia.edu,2.0,'- I started with question 3 because I believe...,5.0,'- Responses on classroom norms were more help...,"'- Typical for high energy ""fidgety"" student.\...",Ethan is high energy. He may be considered to ...,3.0,2.0,2.0,...,5.0,5.0,4.0,4.0,Thinking about wait time. I talked over the s...,I could use help developing ice breakers. I a...,None at this time,1.4,2.0,1.714286
5,jlg2ga@virginia.edu,9.0,'-talked about suspense and our own understand...,10.0,'-we all agreed on the norms\n-the students we...,'-laid back and funny\n-made a comment that hi...,I feel that Ethan is someone you need to work ...,4.0,1.0,1.0,...,5.0,5.0,5.0,5.0,I think that having five different personaliti...,Sometimes it was hard to understand them/tell ...,None!,1.4,2.4,1.928571
6,sdr9x@virginia.edu,5.0,'--I focused more on giving them a way to rela...,7.0,'--I really liked giving them ownership of und...,'-Ethan tested that boundary and responded to ...,'-Prior experience in the classroom\n-Testing ...,2.0,1.0,2.0,...,5.0,5.0,3.0,4.0,'--lower risk at experimenting with techniques...,'--more students. \n--time to finish a conver...,n/a thanks to all who have developed and are c...,2.0,1.8,1.785714
7,jms3ea@virginia.edu,7.0,I think my feedback helped the students in the...,7.0,I think I did a good job helping the kids agre...,'-comfortable - casual -familiar,Ethan may be most comfortable in situations th...,2.0,1.0,1.0,...,3.0,3.0,4.0,4.0,It was beneficial to practice slowing down my ...,I would have been extra comfortable if I had b...,My only minor concern is that I won't be able ...,1.0,1.2,1.071429
8,yk5kb@virginia.edu,6.0,'-I asked students to explain their rationale ...,8.0,"'-Although there was an awkward silence, conve...",'-chill and relaxed\n-sweet,'-He may get off track easily and feel unmotiv...,3.0,3.0,2.0,...,3.0,5.0,3.0,4.0,I got to speak to students and facilitate a lo...,More preparation beforehand,NaN,1.0,2.4,1.714286
9,hnj2gq@virginia.edu,9.0,They seemed to grasp the overall idea of suspe...,10.0,I went around in a circle so that everyone had...,He was respectful and contributed equally with...,I think that he cares about classroom expectat...,2.0,1.0,2.0,...,5.0,5.0,5.0,5.0,It felt more real than I thought it was going ...,I do wish that it was easier to understand wha...,"I don't think I have any concerns, I'm glad we...",1.2,1.4,1.285714
10,krv2td@virginia.edu,3.0,1. I redirected their answers back to the text...,4.0,1. I allowed the students to discuss with each...,'- Ethan was not actively involved\n-Ethan's p...,Ethan might not experience set rules and expec...,3.0,1.0,3.0,...,2.0,2.0,1.0,4.0,I had to improvise and constantly think on my ...,The voices are super creepy. The audio is also...,NaN,1.0,2.4,1.642857
12,ajm8gx@virginia.edu,10.0,'-I think that delving into the background of ...,10.0,'-In disciplining Ethan I was able to make sur...,Ethan was off-task often.\nHe i

In [16]:
Reverse_Approach_Scale(data,app_map)
Generate_App_Scale(data)
data

,email,sim_fbsk,affect,sim_cmsk,score,rationale,sim_txt_eth_contribute,beh_fidgeting,beh_humming,beh_excitable,...,app_confer_stu_rc,app_confer_parent_rc,app_behavior_plan_rc,app_challenge_work_rc,app_spend_time_rc,app_space_regroup_rc,app_beh_manage_coach_rc,app_beh_manage_teach_rc,manage_app_negative,manage_app_positive
2,gct2y@virginia.edu,6.0,A few of the students seemed a bit confused ab...,5.0,"The students provided great ideas, but even th...","Ethan seemed a little bit too cool for school,...","I've seen students like Ethan, and the most su...",3.0,2.0,2.0,...,9.0,8.0,5.0,4.0,3.0,7.0,7.0,3.0,2.8,5.250
3,wmn6kc@virginia.edu,7.0,I think my feedback allowed everyone to voice ...,10.0,"I asked the students as a whole for ideas, and...",Mostly on task. He did go text his mother befo...,Most likely it would have to do with his famil...,2.0,1.0,1.0,...,10.0,8.0,10.0,10.0,5.0,10.0,10.0,10.0,1.6,1.875
4,kby6hf@virginia.edu,2.0,'- I started with question 3 because I believe...,5.0,'- Responses on classroom norms were more help...,"'- Typical for high energy ""fidgety"" student.\...",Ethan is high energy. He may be considered to ...,3.0,2.0,2.0,...,1.0,7.0,9.0,3.0,2.0,1.0,6.0,1.0,5.6,7.250
5,jlg2ga@virginia.edu,9.0,'-talked about suspense and our own understand...,10.0,'-we all agreed on the norms\n-the students we...,'-laid back and funny\n-made a comment that hi...,I feel that Ethan is someone you need to work ...,4.0,1.0,1.0,...,2.0,5.0,1.0,7.0,1.0,1.0,9.0,1.0,3.0,7.625
6,sdr9x@virginia.edu,5.0,'--I focused more on giving them a way to rela...,7.0,'--I really liked giving them ownership of und...,'-Ethan tested that boundary and responded to ...,'-Prior experience in the classroom\n-Testing ...,2.0,1.0,2.0,...,4.0,8.0,5.0,4.0,5.0,4.0,9.0,5.0,3.2,5.500
7,jms3ea@virginia.edu,7.0,I think my feedback helped the students in the...,7.0,I think I did a good job helping the kids agre...,'-comfortable - casual -familiar,Ethan may be most comfortable in situations th...,2.0,1.0,1.0,...,10.0,10.0,10.0,10.0,9.0,9.0,10.0,8.0,2.2,1.500
8,yk5kb@virginia.edu,6.0,'-I asked students to explain their rationale ...,8.0,"'-Although there was an awkward silence, conve...",'-chill and relaxed\n-sweet,'-He may get off track easily and feel unmotiv...,3.0,3.0,2.0,...,7.0,9.0,8.0,8.0,3.0,8.0,9.0,7.0,2.4,3.625
9,hnj2gq@virginia.edu,9.0,They seemed to grasp the overall idea of suspe...,10.0,I went around in a circle so that everyone had...,He was respectful and contributed equally with...,I think that he cares about classroom expectat...,2.0,1.0,2.0,...,10.0,10.0,10.0,9.0,9.0,8.0,10.0,10.0,1.0,1.500
10,krv2td@virginia.edu,3.0,1. I redirected their answers back to the text...,4.0,1. I allowed the students to discuss with each...,'- Ethan was not actively involved\n-Ethan's p...,Ethan might not experience set rules and expec...,3.0,1.0,3.0,...,7.0,9.0,7.0,6.0,2.0,2.0,9.0,8.0,3.2,4.750
12,ajm8gx@virginia.edu,10.0,'-I think that delving into the background of ...,10.0,'-In disciplining Ethan I was able to make sur...,Ethan was off-task often.\nHe interrupted othe...,He may have ADD or another IEP. He may have a ...,4.0,4.0,4.0,...,1.0,1.0,1.0,5.0,1.0,2.0,1.0,1.0,8.2,9.375
